In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os

## In questo file andiamo ad effettuare il clustering con KMEANS per il dataframe contenente tutti i file csv

In [ ]:
with open('pickle/scaled_total_document.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [ ]:
#eliminazione duplicati

df = df.drop_duplicates()

In [ ]:
#contiamo il numero di elementi per ogni label

df_prova = df.groupby(['Label'])['Label'].count()

df_prova=df_prova.to_frame()
print(df_prova)
df_prova.set_index('Label')
df_prova=df_prova.rename(columns={'Label':'Count'})

In [ ]:
# Seleziona una frazione casuale delle righe in base a una colonna specifica
# Selezioniamo il 50% delle righe

colonna_interessata = 'Label'
frazione_da_selezionare = 0.5  # Ad esempio, seleziona il 50% delle righe


df = df.groupby(colonna_interessata).apply(lambda x: x.sample(frac=frazione_da_selezionare, random_state=42)).reset_index(drop=True)

In [ ]:
#contiamo il numero di elementi per ogni label

df_prova = df.groupby(['Label'])['Label'].count()

df_prova=df_prova.to_frame()
print(df_prova)
df_prova.set_index('Label')
df_prova=df_prova.rename(columns={'Label':'Count'})

In [ ]:
#prendiamo due feature alla volta

feature_1 = ''
feature_2 = ''


df = df[[feature_1, feature_2,'Label']]

In [ ]:
malign = df[df.Label == 1]
benign = df[df.Label == 0]

print("Benign: ", len(benign), "Bot: ", len(malign))

In [ ]:
#bilanciamento del dataset
balanced_d = pd.concat([malign, benign.sample(len(malign))])
bal_x = balanced_d.iloc[:,:-1]
bal_y = balanced_d.iloc[:,-1:]
balanced_d.shape

In [ ]:
n_clusters = len(balanced_d['Label'].unique())  # numero di cluster

In [ ]:
# Eseguiamo il clustering con Kmeans

km = KMeans(n_clusters=n_clusters)
y_pred = km.fit_predict(bal_x)

# Associazione del cluster i-esimo con la classe i-esima
cluster_class_mapping = {}
for i in range(n_clusters):
    cluster_samples = bal_x[y_pred == i]
    cluster_classes = bal_y[y_pred == i]
    unique_classes, class_counts = np.unique(cluster_classes, return_counts=True)
    dominant_class = unique_classes[np.argmax(class_counts)]
    cluster_class_mapping[i] = dominant_class

In [ ]:
# Plot dei risultati

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
scatter = axs.scatter(bal_x[feature_1], bal_x[feature_2], c=y_pred)
scatter
axs.set_title("K-means")
plt.xlabel(feature_1)
plt.ylabel(feature_2)

plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1], s = 50, c = 'blue' , label = 'centroid')

difference = pd.DataFrame(data=[],columns=['num_cluster','num_class'])
# Stampiamo l'associazione del cluster con la classe
for i in range(n_clusters):
    if(i != cluster_class_mapping[i]):

        new_row = pd.Series({'num_cluster': i, 'num_class': cluster_class_mapping[i]})
        difference = pd.concat([difference,new_row.to_frame().T],ignore_index=True)
        
    axs.text(
        np.mean(bal_x[feature_1][y_pred == i]), np.mean(bal_x[feature_2][y_pred == i]),
        f"Cluster {i}: Class {cluster_class_mapping[i]}",
        fontsize=12, fontweight='bold', color='red', ha='center', va='center'
    )

title = ("K-means " + str(feature_1) + " " + str(feature_2))
feature_name = title.replace('/','_')
if not os.path.exists('image/clustering_totale/'):
    os.makedirs('image/clustering_totale/')
plt.savefig('image/clustering_totale/'+str(feature_name)+'.png')

plt.show()

In [ ]:
# se il numero del cluster e la classe non coincidono, andiamo ad effettuare uno switch

indici = []

for a in difference['num_cluster']:
    indici.append([
    index for index in range(len(y_pred))
    if y_pred[index] == a
])

for ind,true_value in zip(indici,difference['num_class']):
    for a in ind:
        y_pred[a] = true_value

In [ ]:
accuracy_kmeans = str(accuracy_score(y_pred,bal_y))
precision_kmeans = str(precision_score(y_pred,bal_y))
recall_kmeans = str(recall_score(y_pred,bal_y))

# accuratezza kmeans
print("accuratezza" + accuracy_kmeans)
# precision kmeans
print("precision" + precision_kmeans)
# recall kemans
print("recall" + recall_kmeans)


metriche = pd.DataFrame({
    'metriche': ["accuracy","precision","recall"],
    'valori': [accuracy_kmeans,precision_kmeans,recall_kmeans]
})

if not os.path.exists('metriche/'):
    os.makedirs('metriche/')

metriche.to_csv('metriche/metriche_kmeans_totale.csv',index=False)